In [1]:
import geopandas as gpd

# Load GeoJSON file
geojson_file = 'train.geojson'
gdf = gpd.read_file(geojson_file)


# Save DataFrame to XLSX
xlsx = 'train.xlsx'
gdf.to_excel(xlsx, index=False)

# Load GeoJSON file
geojson_file = 'test.geojson'
gdf = gpd.read_file(geojson_file)


# Save DataFrame to XLSX
xlsx = 'test.xlsx'
gdf.to_excel(xlsx, index=False)

In [ ]:

import import_ipynb
import processing_data
import pandas as pd
df_test = pd.read_excel('test.xlsx')
processing_data.processed_file(df_test,'test_processed')

df_train = pd.read_excel('train.xlsx')
processing_data.processed_file(df_train,'train_processed')

From there, I used models that I thought were good based on the f1 score metric I was using and some grid research. Using these models, I predicted the labels I needed, and in the end I came 6th out of over 130 competitors in the kaggle competition.

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load all your data from the Excel file
X = pd.read_excel('train_processed.xlsx')  # Read training data from Excel

# Separate features and the target variable
X_all = X.drop(columns=['change_type'])  # Drop the target column from features
y_all = X['change_type']  # Extract the target column

# Encode class names into numerical values
label_encoder = LabelEncoder()  # Initialize the label encoder
y_all_encoded = label_encoder.fit_transform(y_all)  # Fit and transform target variable

# Create the XGBoost dataset with all the data
dall = xgb.DMatrix(X_all, label=y_all_encoded)  # Create XGBoost data matrix

# Define model parameters
params = {
    'objective': 'multi:softmax',  # Multi-class classification
    'num_class': len(set(y_all_encoded)),  # Number of classes
    'eval_metric': 'mlogloss',  # Evaluation metric for multi-class log-loss
    'max_depth': 8,  # Maximum depth of trees
    'learning_rate': 0.24  # Learning rate
}

# Train the model with 1695 trees on all the data
num_rounds = 1695  # Number of boosting rounds
bst = xgb.train(params, dall, num_rounds)  # Train the XGBoost model

# Load new data for which you want to make predictions
new_data = pd.read_excel('test_processed.xlsx')  # Load test data from Excel

# Preprocess the new data in the same way as the old data
X_new = new_data  # Assign new data to variable X_new
# No need to transform y_new, as it's what we want to predict

# Create an XGBoost dataset for the new data
dnew = xgb.DMatrix(X_new)  # Create data matrix for the new data

# Make predictions on the new data
y_pred_encoded = bst.predict(dnew)  # Get the encoded predictions

# Convert the encoded predictions back to the original class names
y_pred = label_encoder.inverse_transform(y_pred_encoded.astype(int))  # Convert to class names

# Display the predictions
print("Predictions on the new data:")  # Print the results
print(y_pred)  # Display the predictions

# Dictionary for mapping original class names to specific values
correspondance = {
    'Demolition': 0,
    'Road': 1,
    'Residential': 2,
    'Commercial': 3,
    'Industrial': 4,
    'Mega Projects': 5
}

# Function to map values based on the dictionary
def mapper(valeur):
    return correspondance.get(valeur, valeur)  # Return mapped value or the original value if not in dictionary

# Apply the mapping function to the predictions array
y_pred = np.vectorize(mapper)(y_pred)  # Vectorize the mapping function and apply it

# Save results to a submission file
pred_df = pd.DataFrame(y_pred)  # Create a DataFrame with predictions
pred_df.to_csv("sample_submission1.csv", index=True, index_label='Id')  # Save to CSV with index labeled as 'Id'


In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load the data from an Excel file
data = pd.read_excel('train_processed.xlsx')  # Load the training data

# Separate features and the target variable
X = data.drop(columns=['change_type'])  # Extract the features
y = data['change_type']  # Extract the target variable

# Encode the target variable as numeric values
label_encoder = LabelEncoder()  # Initialize the label encoder
y = label_encoder.fit_transform(y)  # Transform the target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)  # 10% test set

# Create different XGBoost models
model1 = XGBClassifier(max_depth=8, n_estimators=1700, learning_rate=0.25)  # Model 1 with 1700 trees
model2 = XGBClassifier(max_depth=8, n_estimators=1695, learning_rate=0.24)  # Model 2 with 1695 trees
model3 = XGBClassifier(max_depth=8, n_estimators=1705, learning_rate=0.25)  # Model 3 with 1705 trees

# Create a VotingClassifier with the XGBoost models
voting_clf = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')  # Soft voting

# Train the VotingClassifier
voting_clf.fit(X, y)  # Train on the entire dataset

# Load new data for which you want to make predictions
new_data = pd.read_excel('test_processed.xlsx')  # Load test data from Excel

# Preprocess the new data the same way as the old data
X_new = new_data  # Assign the new data to variable X_new
# No need to transform y_new, as that's what we're predicting

# Make predictions on the new data with the VotingClassifier
y_pred_encoded = voting_clf.predict(X_new)  # Get the predictions

# Convert the encoded predictions back to the original class names
y_pred = label_encoder.inverse_transform(y_pred_encoded.astype(int))  # Convert to class names

# Display the predictions
print("Predictions on the new data:")  # Output to console
print(y_pred)  # Print the predictions

# Dictionary for mapping class names to specific values
correspondance = {
    'Demolition': 0,
    'Road': 1,
    'Residential': 2,
    'Commercial': 3,
    'Industrial': 4,
    'Mega Projects': 5
}

# Function to map values based on the dictionary
def mapper(valeur):
    return correspondance.get(valeur, valeur)  # Return the mapped value or the original if not found

# Apply the transformation to the predictions array
y_pred = np.vectorize(mapper)(y_pred)  # Vectorize the mapper function and apply

# Save results to a submission file
pred_df = pd.DataFrame(y_pred)  # Create a DataFrame for the predictions
pred_df.to_csv("sample_submission_voting2.csv", index=True, index_label='Id')  # Save to CSV with index labeled as 'Id'


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the data from an Excel file
data = pd.read_excel('train_processed.xlsx')  # Read the dataset from Excel

# Separate features and the target variable
X = data.drop(columns=['change_type'])  # Drop the target column to get the features
y = data['change_type']  # Extract the target column

# Encode class names into numerical values
label_encoder = LabelEncoder()  # Initialize the label encoder
y = label_encoder.fit_transform(y)  # Transform class names into numeric labels

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% training, 10% test

# Define parameter grid for the grid search
param_grid = {
    'learning_rate': [0.25, 0.24, 0.26],  # Learning rates to test
    'max_depth': [8],  # Fixed depth of 8
    'n_estimators': [1700, 1695],  # Number of trees to test
    'gamma': [0],  # No gamma
    'alpha': [0],  # No alpha
    'lambda': [0]  # No lambda
}

# Create the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y)), seed=42)  # Multi-class with fixed seed

# Grid search to find the best parameters
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=3, scoring='f1_macro')  # 3-fold cross-validation
grid_search.fit(X_train, y_train)  # Fit the grid search on the training data

# Display the best parameters found
print("Best parameters found:")  # Print best parameters
print(grid_search.best_params_)  # Show the best parameters found by grid search

# Predictions on the test set with the best parameters
best_xgb_classifier = grid_search.best_estimator_  # Get the best classifier
y_pred = best_xgb_classifier.predict(X_test)  # Make predictions on the test set

# Calculate F1-score for model accuracy
f1 = f1_score(y_test, y_pred, average='micro')  # Micro-average F1-score
print("Model accuracy with best parameters:", f1)  # Display model accuracy

# Display the classification report
print("Classification report:")  # Output the classification report
print(classification_report(y_test, y_pred))  # Show the detailed report
